In [1]:
# IMPORTS

import rebound as rb
from matplotlib import pyplot as plt
import celmech as cm
import numpy as np
import sympy as sp
import radvel
from tqdm import tqdm
import pandas as pd
import h5py
from scipy import optimize
from multiprocessing import Pool

/home/ichow9/.conda/envs/wmpl/lib/python3.10/site-packages/radvel/gp.py:32: ImportWarning: celerite not installed. GP kernals using celerite will not work. Try installing celerite using 'pip install celerite'
  warnings.warn("celerite not installed. GP kernals using celerite will not work. \


Import the data:

In [2]:
MAY_1_2015 = 57143.5  # barycentric julian date for May 1, 2015 (the date of the HARPS instrument upgrade as per trifonov et al 2020)
# 57143.5 is BJD for May 1, 2015
# 57173.5 is BJD for May 31, 2015

# harps
# hd_data_harps = pd.read_csv('hd45364_rvs.csv', sep = ';')
hd_data_harps = pd.read_csv('HD45364_HARPS_RVBank_ver02.csv', sep=',')  # updated version
# giant outlier at position 116 in the data (found manually earlier) which we remove
hd_data_harps.drop(116, inplace=True)  # drop the row and keep the df in place
# subtract 2.4e6 from all the rows in the data
hd_data_harps.BJD -= 2.4e6
# rename target to HARPS1 or HARPS2
hd_data_harps['target'] = hd_data_harps.apply(lambda row: 'HARPS1' if row.BJD < MAY_1_2015 else 'HARPS2', axis = 1)
# hires
hd_data_hires = pd.read_csv('../hires_rvs.txt', sep = '\t', index_col=False, header='infer', dtype=np.float64)
hd_data_hires['BJD - 2,450,000'] += 50000.  # adding 50000 to have the same units as harps
hd_data_hires['target'] = 'HIRES'
hd_data_hires.columns = ['BJD', 'RV_mlc_nzp', 'e_RV_mlc_nzp', 'target']
# concatenate two data sets one on top of the other
hd_data = pd.concat((hd_data_harps, hd_data_hires), axis=0)  # matching BJD, RV_mlc_nzp and e_RV_mlc_nzp columns
# reset index
hd_data.reset_index(drop=True, inplace=True)

Import posterior distribution for strong $A = 0.1$ penalty:

In [3]:
# import the posterior distribution data for the STRONG PENALTY (A = 0.1)
cluster_data = h5py.File('../mcmc_hd45364_everything_with_libration_penalty_variable_1.h5', 'r')  # import the posterior distribution data
accepted, samples, log_prob = np.array(cluster_data['mcmc']['accepted']), np.array(cluster_data['mcmc']['chain']), np.array(cluster_data['mcmc']['log_prob'])
n_burn_in = 200  # discard the first 200 samples as burn-in time
# reshape the chain to flatten it out
flat_samples = samples[n_burn_in:].reshape(-1, samples[n_burn_in:].shape[-1])

Functions to analytically compute the $K$ value:

In [4]:
#Very original parameters used in Hadden and Payne
nbody_params =[ 2.27798546e+02,  7.25405874e+00,  5.39392010e+04,  1.71866112e-01, 1.17923823e-01,  
               3.43881599e+02,  1.87692753e+01,  5.40138425e+04, 1.68408461e-01,  5.05903191e-02, 
               -3.28526403e-03, 0., 0., 
               1, 
               1.84, 0., 0.]  # inserted 0 for harps2 and hires for both rv offset and jitter

# #Least squares fit: 
# fit_params = [ 2.28512793e+02, 7.27736501e+00, 5.39371914e+04, -4.66868256e-02, 
#                -1.78080009e-01, 3.43378038e+02, 1.78603341e+01, 5.40186750e+04, 
#                9.72945632e-02,  1.32194117e-01, -5.29072002e-01, 0., 0., 1, 2.428]#-7.68527759e-03] 

# Neg log likelihood jitter fit:
prev_params = [ 2.27510047e+02,  7.21459722e+00, 5.39394197e+04, -1.45510376e-02, -1.91998583e-01,
              3.44196007e+02,  1.80943200e+01,  5.47060928e+04, 9.38174624e-02,  1.11054397e-01,
              -1.80048668e-01, -1.44155418e+00, 1.40493043e+00,
              1.00000000e+00,
              1.46046278e+00,  6.96508946e-01, 3.45217643e+00]

# LI ET AL. 2022 PARAMS
li_params = [225.34, 7.26, radvel.orbit.timeperi_to_timetrans(53375, 225.34, 0.07, np.deg2rad(92)), np.sqrt(0.07) * np.cos(np.deg2rad(92)), np.sqrt(0.07) * np.sin(np.deg2rad(92)),
             345.76, 18.17, radvel.orbit.timeperi_to_timetrans(53336, 345.76, 0.01, np.deg2rad(276)), np.sqrt(0.01) * np.cos(np.deg2rad(276)), np.sqrt(0.01) * np.cos(np.deg2rad(276)),
            -1.80048668e-01, -1.44155418e+00, 1.40493043e+00,
              1.00000000e+00,
              1.46046278e+00,  6.96508946e-01, 3.45217643e+00]

# TRIFONOV PARAMS
trifonov_params = [226.57, 7.29, radvel.orbit.timeperi_to_timetrans(52902, 226.57, 0.0796, np.deg2rad(244.44)), np.sqrt(0.0796) * np.cos(np.deg2rad(244.44)), np.sqrt(0.0796) * np.sin(np.deg2rad(244.44)),
              344.66, 18.21, radvel.orbit.timeperi_to_timetrans(52920, 344.66, 0.002, np.deg2rad(20.342)), np.sqrt(0.0002) * np.cos(np.deg2rad(20.342)), np.sqrt(0.0002) * np.sin(np.deg2rad(20.342)),
              0.041, -3.348, 2.708,
              np.sin(np.deg2rad(83.7597)),
              1.437, 0.763, 3.136]

# UPDATED TRIFONOV PARAMS
fit_params = [ 2.27879597e+02,  7.26670196e+00,  5.27997039e+04, -4.33392738e-02,
       -2.20477279e-01,  3.44061366e+02,  1.82074627e+01,  5.29858704e+04,
        9.68685024e-02,  2.30277928e-02, -3.66896966e-02, -3.32570314e+00,
        1.74545697e+00,  1.00000000e+00,  1.40646019e+00,  6.94463892e-01,
        3.00077415e+00]

## CONSTANTS:

# STAR_MASS = 920  # 920 jupiter masses
STAR_MASS = 859
G = 2.825e-7  # converting G to jupiter masses, au, and days
AUDAY_MS = 1.731e6  # conversion factor for au/day to m/s

obs_time_base = np.median(hd_data_harps.BJD)

# print(f'nbody_params:{nbody_params}\n fit_params:{fit_params}')

def mass_to_semiamp(planet_mass, star_mass, period, eccentricity, inclination):
    """
    planet mass (jupiter masses) to semi amplitude (in au/day)
    """
    return ((2 * np.pi * G/period) ** (1/3) * (planet_mass * np.sin(inclination) / star_mass ** (2/3)) * (1/np.sqrt(1 - eccentricity ** 2)))


def semiamp_to_mass(semiamp, star_mass, period, eccentricity, inclination):
    """
    semi amplitude (in au/day) to planet mass (jupiter masses)
    """
    return (((2 * np.pi * G/period) ** (-1/3)) * (semiamp / np.sin(inclination)) * np.sqrt(1 - eccentricity ** 2) * (star_mass ** (2/3)))


def get_sim_from_params(params, integrator, time_base, star_mass = STAR_MASS, auday_ms = AUDAY_MS):
    """
    takes in params array, returns a rebound Simulation object with those parameters
    
    param params: numpy array of params:
    
    for i in range(0, num_planets):
    
    params[i + 0] is period
    params[i + 1] is semiamp
    params[i + 2] is tc (time of conjunction)
    params[i + 3] is sqrt(e) * cos(omega)
    params[i + 4] is sqrt(e) * sin(omega)
    
    params[5 * num_planets] is rv offset for HARPS1
    params[5 * num_planets + 1] is rv offset for HARPS2
    params[5 * num_planets + 2] is rv offset for HIRES
    params[5 * num_planets + 3] is sin(i)
    params[5 * num_planets + 4] is jitter for HARPS1
    params[5 * num_planets + 5] is jitter for HARPS2
    params[5 * num_planets + 6] is jitter for HIRES
    
    param integrator: integrator to use, one of 'whfast' or 'ias15'
    param time_base: base time (to begin integration from) in the simulation
    """
    
    num_planets = 2 # 2 planets
    
    sim = rb.Simulation()
    sim.integrator = integrator
    sim.t = time_base  # keplerian and n-body models initialized at the same time offset
    # print(sim.t)
    if integrator == 'whfast':  # if using whfast integrator, set timestep
        sim.dt = 1/50 * np.min([params[0], params[5]])  # timestep is 1/20th of the shortest orbital period of any planet
        # print(sim.dt)
    sim.units = ('AU', 'Mjupiter', 'day')
    sim.add(m = star_mass)  # star mass as a constant
    
    inclination = np.arcsin(params[-4])  # sin(i) is fourth from the back of the array
        
    for i in range (0, num_planets):
        # print(i)
        # planet parameters
        period = params[5*i]  # in days
        semiamp = params[5*i + 1] / auday_ms # divide by auday_ms because semiamp given in m/s
        eccentricity = params[5*i + 3] ** 2 + params[5*i + 4] ** 2  # eccentricity from secos, sesin
        omega = np.arctan2(params[5*i + 4], params[5*i + 3])  # omega from arctan of sesin, secos  (in that order!)
        # get tp by converting from tc
        tp = radvel.orbit.timetrans_to_timeperi(tc = params[5*i + 2], per = period, ecc = eccentricity, omega = omega)
        
        # mass
        mass = semiamp_to_mass(semiamp = semiamp, star_mass = star_mass, period = period, eccentricity = eccentricity, inclination = inclination)
        
        # adding to simulation
        sim.add(m = mass, P = period, e = eccentricity, T = tp, omega = omega, inc = inclination)
        
    sim.move_to_com()  # move to center of mass
    
    return sim

def get_simple_sim(masses, integrator = 'ias15', period_ratio = 3/2, epsilon=0.01):
    """
    gets simple sim (for eccentricity track stuff)
    param masses: array of planet masses
    param integrator: integrator
    param epsilon: amount by which the resonant period ratio should be offset from the equilibrium in the simulation
    """
    sim = rb.Simulation()
    sim.integrator = integrator
    # central star
    sim.add(m = 1)
    
    sim.add(m = masses[0], P = 1)
    sim.add(m = masses[1], P = period_ratio * (1 + epsilon))

    sim.move_to_com()
    if integrator == 'whfast':
        sim.dt = 1/50 * 1  # dy default use 1/50th of the inner planet's orbital period for the timestep if using whfast
    return sim


def get_rvs(params, instrument, times, integrator, time_base, auday_ms = AUDAY_MS):
    
    """
    Gets RVs from a Numpy array of planet params
    
    param params:     for i in range(0, num_planets):
    
    params[i + 0] is period
    params[i + 1] is semiamp
    params[i + 2] is tc (time of conjunction)
    params[i + 3] is sqrt(e) * cos(omega)
    params[i + 4] is sqrt(e) * sin(omega)
    
    params[5 * num_planets] is rv offset for HARPS1
    params[5 * num_planets + 1] is rv offset for HARPS2
    params[5 * num_planets + 2] is rv offset for HIRES
    params[5 * num_planets + 3] is sin(i) (also params[-4])
    params[5 * num_planets + 4] is jitter for HARPS1 (also params[-3])
    params[5 * num_planets + 5] is jitter for HARPS2 (also params[-2])
    params[5 * num_planets + 6] is jitter for HIRES (also params[-1])

    param instrument: instrument (HARPS1, HARPS2, or HIRES)
    param times: array of times to integrate over
    param integrator: integrator to use, one of 'whfast' or 'ias15'
    
    """
    
    sim = get_sim_from_params(params, integrator, time_base = time_base)
    
    sim_backwards = sim.copy()
    sim_backwards.dt *= -1  # set timestep to be negative if integrating backwards

    times = pd.Series(times)  # convert to series if not already
    
    forward_times = times[times - time_base >= 0]
    backward_times = times[times - time_base < 0]
    forward_indices = forward_times.index
    backward_indices = backward_times.index
    
    # initialize rvs
    rv_forward = np.zeros(len(forward_times))
    rv_backward = np.zeros(len(backward_times))
    
    num_planets = 2  # find number of planets in params passed
    
    # get the rvs (z velocity, assuming 90 deg inclination) from the rebound simulation to compare with the actual simulation
    for j, it in enumerate(zip(forward_indices, forward_times)):
        i, t = it  # forward index, forward time
        sim.integrate(t, exact_finish_time = 1)
        # integrate to the specified time, exact_finish_time = 1 for ias15, 
        # sim.status()
        star = sim.particles[0]
        # print(instrument[i])
        # use one of 3 different radial velocity offsets depending on whether the data is from HARPS1, HARPS2 or HIRES
        if instrument[i] == 'HARPS1':
            rv_offset = params[5 * num_planets]
        elif instrument[i] == 'HARPS2':
            rv_offset = params[5 * num_planets + 1]
        elif instrument[i] == 'HIRES':
            rv_offset = params[5 * num_planets + 2]
        else:
            rv_offset = 0.
        rv_forward[j] = (-star.vz * auday_ms) + rv_offset  # use x-velocity of the star as the radial velocity, convert to m/s
    
    for j, it in enumerate(zip(backward_indices, backward_times)):
        i, t = it  # backward index, backward time
        sim_backwards.integrate(t, exact_finish_time = 1)
        star = sim_backwards.particles[0]
        # use one of 3 different radial velocity offsets depending on whether the data is from HARPS1, HARPS2 or HIRES
        # print(instrument[i])
        if instrument[i] == 'HARPS1':
            rv_offset = params[5 * num_planets]
        elif instrument[i] == 'HARPS2':
            rv_offset = params[5 * num_planets + 1]
        elif instrument[i] == 'HIRES':
            rv_offset = params[5 * num_planets + 2]
        else:
            rv_offset = 0.
        rv_backward[j] = (-star.vz * auday_ms) + rv_offset
    
    return np.concatenate((rv_backward, rv_forward))

Functions to compute the best-fit:

In [5]:
def get_nbody_resids(params, integrator, data = hd_data):
    """
    Gets the normalized residuals for the n-body fit with REBOUND
    """
    obs_y = data.RV_mlc_nzp  # observed RVs
    synth_y = get_rvs(params, data.target, data.BJD, integrator, time_base=obs_time_base)  # RVs from the rebound simulation
    obs_yerr = data.e_RV_mlc_nzp  # y errors
    return (obs_y - synth_y) / obs_yerr  # return normalized residuals


def neg_log_likelihood(params, time_base = obs_time_base, data = hd_data, num_planets = 2):
    """
    Gets the negative log-likelihood (including a jitter term!) for use with scipy.optimize.minimze
    
    Iplements the log likelihood using the same method above
    
    """
    obs_y = data.RV_mlc_nzp  # observed RVs
    
    # inclination not handled sparately
    # inclination = np.arcsin(params[-4])  # inclination is np.arcsin of the second to last parameter
    
    synth_y = get_rvs(params, data.target, data.BJD, 'ias15', time_base = time_base)  # RVs from the rebound simulation
    obs_yerr = data.e_RV_mlc_nzp  # y errors

    conditions = [data.target == 'HARPS1', data.target == 'HARPS2', data.target == 'HIRES']  # conditions are harps1, harps2 or hires

    rv_offsets = params[5 * num_planets:5 * num_planets + 3]  # rv_offsets for HARPS1, HARPS2 and HIRES, in that order
    jitters = params[-3:]  # jitters for HARPS1, HARPS2 and HIRES, in that order
    
    # get the jitter and rv values for the corresponding data points
    rv_offset = np.select(conditions, rv_offsets, default=np.nan)
    jitter = np.select(conditions, jitters, default=np.nan)
    # print(rv_offset, jitter)

    # compute the log-likelihood
    #### OLD
    # log_likelihood = -1/2 * np.sum(((obs_y - synth_y) ** 2)/(obs_yerr ** 2 + jitter ** 2) 
    #                                + np.log(np.sqrt(2 * np.pi * (obs_yerr ** 2 + jitter ** 2))))

    #### LI ET AL. 2022 VERSION (NEW)
#     sigma_z2 = 1/(np.sum(1/(obs_yerr ** 2 + jitter ** 2)))
    log_likelihood = -1/2 * np.sum(((obs_y - synth_y) ** 2)/((obs_yerr ** 2 + jitter ** 2))) - np.sum(np.log(np.sqrt(2 * np.pi * (obs_yerr ** 2 + jitter ** 2)))) # + np.log(np.sqrt(2 * np.pi * sigma_z2))
    # log_likelihood = -1/2 * np.sum(np.log(variance) + ((obs_y - synth_y) ** 2/variance))    
    # print(-1/2 * np.sum(((obs_y - rv_offset - synth_y) ** 2)/((obs_yerr ** 2 + jitter ** 2))))
    # print(-log_likelihood)
    return -log_likelihood  # negative since we are trying to minimize the negative log likelihood


def get_tau_alphas(tau_alpha, m_inner, m_outer, period_ratio):
    # use Kepler's third law to compute the ratio of semi-major axes in resonance from the period ratio in resonance
    sma_ratio = period_ratio ** (2 / 3)  # ratio of outer planet's semi-major axis to inner
    # define matrix A
    A = np.array([[-1, 1],
                  [m_outer, m_inner * sma_ratio]])
    # compute gamma_1 and gamma_2
    gammas = np.matmul(np.linalg.inv(A), np.array([-1 / tau_alpha, 0]))
    # gamma = 1/tau
    taus = 1 / gammas

    return tuple(taus)  # returns (tau_alpha_outer, tau_alpha_inner) as a tuple

Function computing $K$ from $D$:

\begin{multline}
     \frac{d}{dt}\mathcal{D}\bigg|_\mathrm{dis} =  0
     \\
     \approx - \frac{\beta_1\sqrt\alpha e_b^2}{\tau_{e, 1}} - \frac{\beta_2 e_2^2}{\tau_{e, 2}} + \frac{\beta_1\beta_2\sqrt{\alpha}}{3\left(j\beta_b\sqrt{\alpha} + (j-1)\beta_c\right)}\frac{3}{2\tau_\alpha},
\end{multline}

$e_i$ are the planets' eccentricities, $\beta_i=m_i/(m_1+m_2)$, $\alpha \approx \left(\frac{j-1}{j}\right)^{2/3}$ is the pair's semi-major axis ratio, and $\Delta = \frac{j-1}{j}\frac{P_2}{P_1} - 1$ measures the pair's fractional deviation from exact period ratio commensurability.

In [6]:
beta_1, beta_2, alpha, e1, e2, tau_e1, tau_e2, j, tau_alpha, tau_e, K = sp.symbols('beta_1, beta_2, alpha, e_1, e_2, tau_e_1, tau_e_2, j, tau_alpha, tau_e, K')

ddot_dis = -(beta_1 * sp.sqrt(alpha) * (e1 ** 2))/(tau_e1) - (beta_2 * (e2) ** 2)/(tau_e2) + (beta_1 * beta_2 * sp.sqrt(alpha))/(3 * (j *  beta_1 * sp.sqrt(alpha) + (j - 1) * beta_2)) * (3/(2 * tau_alpha))
# j and alpha are always the same
j_val = 3.  # 3:2 MMR
alpha_val = ((j_val - 1)/(j_val)) ** (2./3.)
ddot_dis_eq = ddot_dis.subs([(alpha, alpha_val), (j, j_val)])

In [7]:
ddot_dis_eq

                                                               2        2
       1.31037069710445⋅β₁⋅β₂           0.873580464736299⋅β₁⋅e₁    β₂⋅e₂ 
───────────────────────────────────── - ──────────────────────── - ──────
τₐₗₚₕₐ⋅(7.86222418262669⋅β₁ + 6.0⋅β₂)             τₑ ₁              τₑ ₂ 

In [8]:
def ecc_con1(params):
    return 1 - (params[3] ** 2 + params[4] ** 2)

def ecc_con2(params):
    return 1 - (params[8] ** 2 + params[9] ** 2)

cons = ({'type': 'ineq', 'fun': ecc_con1}, 
        {'type': 'ineq', 'fun': ecc_con2})

# bounds of (0, 1) for sin(i), everything else can vary however
bounds = ((None, None), (None, None), (None, None), (None, None), (None, None), 
          (None, None), (None, None), (None, None), (None, None), (None, None), 
          (None, None), (None, None), (None, None),
          (0, 1), 
          (None, None), (None, None), (None, None))
#### BFGS and L-BFGS do not work without hacky fixes (returns RV array full of nans (ecc greater than 1))
#### Nelder-Mead seems to work...
best_fit_jitter = optimize.minimize(neg_log_likelihood, x0=np.array(fit_params), method='Nelder-Mead', 
                                    bounds=bounds, constraints=cons, options={'maxiter': 1000000000000, 
                                                                              'maxfev': 1000000000000, 
                                                                              # 'ftol': 1.e-7
                                                                             }
                                   )  # optimization
# best fit parameters
best = best_fit_jitter.x
best

array([ 2.27879590e+02,  7.26670160e+00,  5.27997040e+04, -4.33386844e-02,
       -2.20477255e-01,  3.44061367e+02,  1.82074511e+01,  5.29858704e+04,
        9.68672855e-02,  2.30292860e-02, -3.66891824e-02, -3.32569568e+00,
        1.74545959e+00,  1.00000000e+00,  1.40646262e+00,  6.94455499e-01,
        3.00076641e+00])

In [9]:
sim = get_sim_from_params(best, integrator='ias15', time_base=obs_time_base)
inner = sim.particles[1]
outer = sim.particles[2]
# get values for e and beta
e1_val, e2_val = inner.e, outer.e
beta_1_val, beta_2_val = inner.m/(inner.m + outer.m), outer.m/(inner.m + outer.m)
# substitute in all the values for ei and betai, as well as 2 * tau_e = tau_e1 = tau_e2 and tau_a = K * tau_e 
ddot_dis_eq_values = ddot_dis_eq.subs([(e1, e1_val), (e2, e2_val), (beta_1, beta_1_val), (beta_2, beta_2_val), (tau_e1, 2 * tau_e), (tau_e2, 2 * tau_e)]).subs(tau_alpha, K * tau_e)
# solve
K_val = sp.solve(ddot_dis_eq_values, K)[0]
np.float64(K_val)

119.59133819696733

Convert $D$ to $K$:

In [10]:
def D_to_K(sample):
    """
    Computes K = tau_a/tau_e for a set of parameters
    """
    i, params = sample  # unpack
    sim = get_sim_from_params(params, integrator='ias15', time_base=obs_time_base)
    inner = sim.particles[1]
    outer = sim.particles[2]
    # get values for e and beta
    e1_val, e2_val = inner.e, outer.e
    beta_1_val, beta_2_val = inner.m/(inner.m + outer.m), outer.m/(inner.m + outer.m)
    # get tau_e, tau_a, tau
    ddot_dis_eq_values = ddot_dis_eq.subs([(e1, e1_val), (e2, e2_val), (beta_1, beta_1_val), (beta_2, beta_2_val), (tau_e1, 2 * tau_e), (tau_e2, 2 * tau_e)]).subs(tau_alpha, K * tau_e)
    # solve
    K_val = np.float64(sp.solve(ddot_dis_eq_values, K)[0])
    return K_val

In [11]:
flat_samples.shape

(2490000, 17)

In [12]:
# create parameter list to parallelize over
par_list = [(i, sample) for i, sample in enumerate(flat_samples)]

# parallelize it and process (takes about 90 minutes)
with Pool() as pool_K_hist:
    print('mapping...')
    K_values = np.array(list(tqdm(pool_K_hist.imap(D_to_K, par_list), total=len(flat_samples)))).astype(np.float64)
    print('done')

# save!
np.save('K_value_array_variable_01_paper_version', K_values)  # save the K values

mapping...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2490000/2490000 [44:33<00:00, 931.32it/s]


done
